In [191]:
import pandas as pd
from sklearn import linear_model
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn import tree

In [192]:
data = pd.read_pickle('train_data.pkl')

In [193]:
data.columns

Index(['DriverNumber', 'Time_1', 'TyreLife_1', 'LapPercent_1', 'Compound_1',
       'LapTime_1', 'Team', 'Driver', 'Time_1_1', 'TyreLife_1_1',
       'LapPercent_1_1', 'Compound_1_1', 'LapTime_1_1', 'Time_2', 'TyreLife_2',
       'LapPercent_2', 'Compound_2', 'LapTime_2', 'Time_1_2', 'TyreLife_1_2',
       'LapPercent_1_2', 'Compound_1_2', 'LapTime_1_2', 'Position_curr',
       'LapPercent_curr', 'Position_prev', 'LapPercent_prev', 'RoundNumber',
       'Country', 'Location', 'OfficialEventName', 'EventDate', 'EventName',
       'EventFormat', 'Compound_1_', 'Compound_1_HARD',
       'Compound_1_INTERMEDIATE', 'Compound_1_MEDIUM', 'Compound_1_SOFT',
       'Compound_1_1_', 'Compound_1_1_HARD', 'Compound_1_1_INTERMEDIATE',
       'Compound_1_1_MEDIUM', 'Compound_1_1_SOFT', 'Compound_1_1_TEST_UNKNOWN',
       'Compound_1_1_WET', 'Compound_2_', 'Compound_2_HARD',
       'Compound_2_INTERMEDIATE', 'Compound_2_MEDIUM', 'Compound_2_SOFT',
       'Compound_2_TEST_UNKNOWN', 'Compound_2_WET', '

In [194]:
data = data.dropna()

In [195]:
data.columns

Index(['DriverNumber', 'Time_1', 'TyreLife_1', 'LapPercent_1', 'Compound_1',
       'LapTime_1', 'Team', 'Driver', 'Time_1_1', 'TyreLife_1_1',
       'LapPercent_1_1', 'Compound_1_1', 'LapTime_1_1', 'Time_2', 'TyreLife_2',
       'LapPercent_2', 'Compound_2', 'LapTime_2', 'Time_1_2', 'TyreLife_1_2',
       'LapPercent_1_2', 'Compound_1_2', 'LapTime_1_2', 'Position_curr',
       'LapPercent_curr', 'Position_prev', 'LapPercent_prev', 'RoundNumber',
       'Country', 'Location', 'OfficialEventName', 'EventDate', 'EventName',
       'EventFormat', 'Compound_1_', 'Compound_1_HARD',
       'Compound_1_INTERMEDIATE', 'Compound_1_MEDIUM', 'Compound_1_SOFT',
       'Compound_1_1_', 'Compound_1_1_HARD', 'Compound_1_1_INTERMEDIATE',
       'Compound_1_1_MEDIUM', 'Compound_1_1_SOFT', 'Compound_1_1_TEST_UNKNOWN',
       'Compound_1_1_WET', 'Compound_2_', 'Compound_2_HARD',
       'Compound_2_INTERMEDIATE', 'Compound_2_MEDIUM', 'Compound_2_SOFT',
       'Compound_2_TEST_UNKNOWN', 'Compound_2_WET', '

In [196]:
train_data = data[data['EventDate'] <= np.datetime64('2022-12-31')]
test_data = data[data['EventDate'] > np.datetime64('2022-12-31')]

In [197]:
X_cols =['TyreLife_1', 'LapPercent_1', 'TyreLife_1_1',
       'LapPercent_1_1', 'TyreLife_2',
       'LapPercent_2', 'TyreLife_1_2',
       'LapPercent_1_2', 
       'Compound_1_HARD', 'Compound_1_INTERMEDIATE', 'Compound_1_MEDIUM',
       'Compound_1_SOFT', 'Compound_1_1_HARD',
       'Compound_1_1_INTERMEDIATE', 'Compound_1_1_MEDIUM', 'Compound_1_1_SOFT', 'Compound_1_1_WET',
       'Compound_2_HARD', 'Compound_2_INTERMEDIATE', 'Compound_2_MEDIUM',
       'Compound_2_SOFT', 'Compound_2_WET',
       'Compound_1_2_HARD', 'Compound_1_2_INTERMEDIATE',
       'Compound_1_2_MEDIUM', 'Compound_1_2_SOFT',
       'Compound_1_2_WET', 'Position_prev', 'LapPercent_prev']

In [198]:
train_X = train_data[X_cols]
train_y = train_data['LapPercent_curr']

In [199]:
train_X.columns.size

29

In [200]:
train_X

,TyreLife_1,LapPercent_1,TyreLife_1_1,LapPercent_1_1,TyreLife_2,LapPercent_2,TyreLife_1_2,LapPercent_1_2,Compound_1_HARD,Compound_1_INTERMEDIATE,...,Compound_2_MEDIUM,Compound_2_SOFT,Compound_2_WET,Compound_1_2_HARD,Compound_1_2_INTERMEDIATE,Compound_1_2_MEDIUM,Compound_1_2_SOFT,Compound_1_2_WET,Position_prev,LapPercent_prev
0,2.0,0.012811,4.0,0.022924,2.0,0.017232,2.0,0.023233,0,0,...,0,1,0,0,0,0,1,0,12.0,0.021704
1,2.0,0.000000,5.0,0.007400,2.0,0.003409,2.0,0.006708,0,0,...,0,1,0,0,0,0,1,0,6.0,0.014633
2,5.0,0.014954,2.0,0.014969,2.0,0.016431,2.0,0.029768,0,0,...,0,1,0,0,0,1,0,0,7.0,0.015634
3,2.0,0.005303,8.0,0.008155,2.0,0.009127,2.0,0.012975,0,0,...,0,1,0,0,0,0,1,0,9.0,0.016143
5,8.0,0.009496,5.0,0.010699,2.0,0.012205,2.0,0.020704,0,0,...,0,1,0,0,0,0,1,0,5.0,0.012822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,2.0,0.014521,5.0,0.014660,2.0,0.010476,5.0,0.011380,0,0,...,0,1,0,0,0,0,1,0,10.0,0.015879
1028,2.0,0.000000,5.0,0.006268,5.0,0.007223,2.0,0.008022,0,0,...,0,1,0,0,0,0,1,0,3.0,0.003973
1029,2.0,0.007330,5.0,0.015964,2.0,0.014669,8.0,0.027435,0,0,...,0,1,0,0,0,1,0,0,17.0,0.033995
1030,2.0,0.002539,5.0,0.014036,3.0,0.004005,8.0,0.013800,0,0,...,0,1,0,0,0,1,0,0,2.0,0.003909


In [231]:
model = tree.DecisionTreeRegressor()

In [233]:
params = {'max_depth':[2,3,4,5,6]}
best_model = GridSearchCV(model, param_grid = params).fit(train_X, train_y)

In [234]:
best_model.best_params_

{'max_depth': 2}

In [235]:
test_X = test_data[X_cols]
test_y = test_data['LapPercent_curr']

In [236]:
pred = best_model.predict(test_X)

In [237]:
np.unique(pred)

array([0.00570954, 0.01572417, 0.02580218, 0.07482009])

In [238]:
results = pd.DataFrame({'pred': pred, 'actual': test_y})

In [239]:
results

,pred,actual
1040,0.015724,0.000000
1041,0.015724,0.027567
1042,0.005710,0.001538
1043,0.005710,0.007000
1044,0.015724,0.003255
...,...,...
1186,0.025802,0.058647
1187,0.025802,0.020070
1189,0.015724,0.014366
1190,0.005710,0.039327


In [240]:
results.describe()

,pred,actual
count,139.000000,139.000000
mean,0.019491,0.017331
std,0.008174,0.038710
min,0.005710,0.000000
25%,0.015724,0.007531
50%,0.015724,0.013350
75%,0.025802,0.019441
max,0.074820,0.455956


In [241]:
joblib.dump(best_model, 'model.pkl')

['model.pkl']